In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from multiprocessing import Pool
from timeit import default_timer as timer
from anomaly_fxns import get_dataframes 
import tqdm
from istarmap import istarmap
from anomaly_fxns import make_merge_dfs
from anomaly_fxns import merge_and_calc_ppt

In [2]:
# nar = pd.read_csv("C:\\Tara_Fall_2019\\Kenya_Drought\\random_samples\\narok_points_completed.csv").replace(-9999.000000, np.NaN)
# tur = pd.read_csv("C:\\Tara_Fall_2019\\Kenya_Drought\\random_samples\\turkana_points_completed.csv").replace(-9999.000000, np.NaN)

nar = pd.read_csv("F:\\Tara_Fall_2019\\Kenya_Drought\\narpts_modis_chirp_chirt_sg.csv")
tur = pd.read_csv("F:\\Tara_Fall_2019\\Kenya_Drought\\turpts_modis_chirp_chirt_sg.csv")

## MODIS new engineering

In [4]:
# create empty output dataframes
nar_out = nar[['FID', "Lat", "Lon"]]
tur_out = tur[['FID', "Lat", "Lon"]]

In [11]:
#weighted average of soil columns
#pass depth as a string value, either 30, 60 or 100
def weighted_avg_tex_new(df, depth, var):
    #pull all columns 
    all_cols = df.columns 
    #pull tex columns 
    var_cols = [col for col in all_cols if var in col]
    if var == "soc":
        new_col = (5/100)*df[(var+'_0-5cm_mean')] + (10/100)*df[(var+'_5-15cm_mean')] + (15/100)*df[(var+'_15-30cm_mean')] + (30/100)*df[(var+'_30-60cm_mean')] + (40/100)*df[(var+'_60-100cm_mean')]
    else: 
        new_col = (5/100)*df[(var+'_0-5cm_mean')] + (10/100)*df[(var+'_5-15cm_mean')] + (15/100)*df[(var+'_15-30cm_mean')] + (30/100)*df[(var+'_30-60cm_mean')] + (40/100)*df[(var+'_60-100cm_mean')]
    #return a series
    return (new_col)

In [12]:
# create full soil profile 
# loop over soil variables and depths
for var in ['sand', 'silt', 'clay', 'soc', 'bdod', 'cfvo']:
    for depth in ['100']:
        for df in [nar, tur]:
            df[(var + depth)] = weighted_avg_tex_new(df,depth,var)
            print (depth, var, " done.")

100 sand  done.
100 sand  done.
100 silt  done.
100 silt  done.
100 clay  done.
100 clay  done.
100 soc  done.
100 soc  done.
100 bdod  done.
100 bdod  done.
100 cfvo  done.
100 cfvo  done.


In [34]:
# Calculate mean annual temp 
years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016"]
for df in [nar, tur]:
    for year in years: 
        T_cols = [col for col in df.columns if "T" in col]
        year_measures = [col for col in T_cols if year in col]
        df[(year + "_avg_temp")] = df[year_measures].mean(axis = 1)

In [39]:
NPP_all = [col for col in nar.columns if "NPP" in col ]
NPP_tur = [col for col in NPP_all if ".1" not in col]
NPP_nar = [col for col in NPP_all if ".1" in col ]

In [41]:
# drop NPP columns from appropriate dataframes 
nar = nar.drop(NPP_tur, axis = 1)
tur = tur.drop(NPP_nar, axis = 1)

In [86]:
# calculate above ground biomass 
# AB = NPP * fANPP * treecovermultiplier  * slopesmultiplier 
def calculate_AB(row, year): 
    columns = list(row.index)
    #pull NPP for a given year
    NPP_col = [col for col in columns if "NPP" in col and year in col][0]
    # pull mean annual temp for given year 
    temp_col= [col for col in columns if "_avg_temp" in col and year in col][0]
    # pull LULC  
    lulc = row.LULC
    # pull slope 
    slope = row.slope
    
    
    # NPP 
    NPP = row[NPP_col]
    # mean annual temperature 
    MAT = row[temp_col]
    # fraction aboveground npp
    fANPP = (0.171) + (0.0129)*MAT 
    
    tree_cover_mult = 1
    # tree cover multiplier - this may need to be toggled  
    if lulc == 1: 
        tree_cover_mult = 0
    else: 
        tree_cover_mult = 1

    # slope multiplier 
    slope_multiplier = 1
    if slope <= 10: 
        slope_multiplier = 1
    elif (slope > 10) & (slope <= 30):
        slope_multiplier = .7
    elif (slope > 30) & (slope <= 60):
        slope_multiplier = .4 
    elif slope > 60:
        slope_multiplier = 0 
    
    
    # above ground biomass 
    AB = NPP * fANPP * tree_cover_mult * slope_multiplier
    
    
    return (AB)

In [88]:
years_t = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016"]
for y in years_t: 
    col_new = y + '_AB'
    tur[col_new] = tur.apply(calculate_AB, year = y, axis = 1)
    print (y, " done.")
    
years_n = ["2000", "2001", "2002", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016"]
for y in years_n: 
    col_new = y + '_AB'
    tur[col_new] = tur.apply(calculate_AB, year = y, axis = 1)
    print (y, " done.")

2000  done.
2001  done.
2002  done.
2003  done.
2004  done.
2005  done.
2006  done.
2007  done.
2008  done.
2009  done.
2010  done.
2011  done.
2012  done.
2013  done.
2014  done.
2015  done.
2016  done.
2000  done.
2001  done.
2002  done.
2005  done.
2006  done.
2007  done.
2008  done.
2009  done.
2010  done.
2011  done.
2012  done.
2013  done.
2014  done.
2015  done.
2016  done.


In [106]:
nar.columns

Index(['Unnamed: 0', 'FID', 'CID', 'Lat', 'Lon', '2000.01.02_8dayppt',
       '2000.01.10_8dayppt', '2000.01.18_8dayppt', '2000.01.26_8dayppt',
       '2000.02.03_8dayppt',
       ...
       '2008_avg_temp', '2009_avg_temp', '2010_avg_temp', '2011_avg_temp',
       '2012_avg_temp', '2013_avg_temp', '2014_avg_temp', '2015_avg_temp',
       '2016_avg_temp', '2000_AB'],
      dtype='object', length=2641)

In [108]:
tur = tur.drop(['Unnamed: 0.1', 'Unnamed: 0', 'FID', 'CID'], axis = 1)
nar = nar.drop([ 'Unnamed: 0', 'FID', 'CID'], axis = 1)

In [111]:
tur.to_csv("F:\\Tara_Fall_2019\\Kenya_Drought\\turkana_AB_analysis.csv")
nar.to_csv("F:\\Tara_Fall_2019\\Kenya_Drought\\narok_AB_analysis.csv")

## Data Quality Control

In [4]:
# fix NDVI columns 
def fix_ndvi(df): 
    #pull all columns 
    all_cols = df.columns
    #ndvi_columns 
    ndvi_cols = [col for col in all_cols if "F" in col and 'ppt' not in col]
    #loop over ndvi columns to mask values outside -1 to 1 with nan
    for col in ndvi_cols: 
        df[col].mask(~df[col].between(-1, 1), inplace=True)

In [5]:
# fix dtype
tur = tur.astype('float64')
nar = nar.astype('float64')

In [6]:
#apply ndvi correction function to both dataframes
fix_ndvi(nar)
fix_ndvi(tur)

In [8]:
#weighted average of soil columns
#pass depth as a string value, either 30, 60 or 100
def weighted_avg_tex(df, depth, var):
    #pull all columns 
    all_cols = df.columns 
    #pull tex columns 
    var_cols = [col for col in all_cols if var in col]
    if var == "soc":
        if depth == '30': 
            new_col = (5/30)*df[(var+'_0_5cm')] + (10/30)*df[(var+'_5_15cm')] + (15/30)*df[(var+'_15_30c')]
        elif depth == '60': 
            new_col = (5/60)*df[(var+'_0_5cm')] + (10/60)*df[(var+'_5_15cm')] + (15/60)*df[(var+'_15_30c')] + (30/60)*df[(var+'_30_60c')]
        elif depth == '100': 
            new_col = (5/100)*df[(var+'_0_5cm')] + (10/100)*df[(var+'_5_15cm')] + (15/100)*df[(var+'_15_30c')] + (30/100)*df[(var+'_30_60c')] + (40/100)*df[(var+'_60_100')]
    else: 
        if depth == '30': 
            new_col = (5/30)*df[(var+'_0_5cm')] + (10/30)*df[(var+'_5_15c')] + (15/30)*df[(var+'_15_30')]
        elif depth == '60': 
            new_col = (5/60)*df[(var+'_0_5cm')] + (10/60)*df[(var+'_5_15c')] + (15/60)*df[(var+'_15_30')] + (30/60)*df[(var+'_30_60')]
        elif depth == '100': 
            new_col = (5/100)*df[(var+'_0_5cm')] + (10/100)*df[(var+'_5_15c')] + (15/100)*df[(var+'_15_30')] + (30/100)*df[(var+'_30_60')] + (40/100)*df[(var+'_60_10')]
    #return a series
    return (new_col)

In [8]:
# loop over soil variables and depths
for var in ['sand', 'silt', 'clay', 'soc', 'bdod', 'cfvo']:
    for depth in ['30', '60', '100']:
        for df in [nar, tur]:
            df[(var + depth)] = weighted_avg_tex(df,depth,var)
            print (depth, var, " done.")

30 sand  done.
30 sand  done.
60 sand  done.
60 sand  done.
100 sand  done.
100 sand  done.
30 silt  done.
30 silt  done.
60 silt  done.
60 silt  done.
100 silt  done.
100 silt  done.
30 clay  done.
30 clay  done.
60 clay  done.
60 clay  done.
100 clay  done.
100 clay  done.
30 soc  done.
30 soc  done.
60 soc  done.
60 soc  done.
100 soc  done.
100 soc  done.
30 bdod  done.
30 bdod  done.
60 bdod  done.
60 bdod  done.
100 bdod  done.
100 bdod  done.
30 cfvo  done.
30 cfvo  done.
60 cfvo  done.
60 cfvo  done.
100 cfvo  done.
100 cfvo  done.


In [9]:
# drop all columns which are mssing texture data 
tur = tur.drop(tur[tur['sand30'] == 0].index)
nar = nar.drop(nar[nar['sand30'] == 0].index)

In [10]:
tur.to_csv("turkana_raw_w_soils.csv")
nar.to_csv("narok_raw_w_soils.csv")

## Get anomalies NEW

In [2]:
# import rangeland data 
tur = pd.read_csv("turkana_raw_w_soils.csv")
nar = pd.read_csv("narok_raw_w_soils.csv")

In [3]:
# pull ndvi columns for turkana and narok
ndvi_cols_t = [col for col in tur.columns if "F" in col and 'ppt' not in col]
ndvi_cols_n = [col for col in nar.columns if "F" in col and 'ppt' not in col]

In [4]:
# get the dates in the format of precipitation columns
ndvi_dates_t = [col[1:5] + "_"+ col[5:7]+"_"+col[7:] for col in ndvi_cols_t]
ndvi_dates_n = [col[1:5] + "_"+ col[5:7]+"_"+col[7:] for col in ndvi_cols_n]

In [5]:

ppt_col_list = [col for col in tur.columns if "ppt" in col]


In [6]:
# create bones for new ppt dataframes
precip_df_t = pd.DataFrame(ndvi_cols_t, columns = ['Date'])
precip_df_t['ppt_date'] = ndvi_dates_t

precip_df_n = pd.DataFrame(ndvi_cols_n, columns = ['Date'])
precip_df_n['ppt_date'] = ndvi_dates_n

In [7]:
# calculate mean NDVI for each row 
tur['mean_NDVI'] = tur[ndvi_cols_t].mean(axis = 1)
nar['mean_NDVI'] = nar[ndvi_cols_n].mean(axis = 1)
tur['stdv_NDVI'] = tur[ndvi_cols_t].std(axis = 1)
nar['stdv_NDVI'] = nar[ndvi_cols_n].std(axis = 1)

In [11]:
tur_by_date = pd.melt(tur, id_vars=['Unnamed: 0'], value_vars=ndvi_cols_t, var_name='Date', value_name='NDVI_value').dropna()
nar_by_date = pd.melt(nar, id_vars=['Unnamed: 0'], value_vars=ndvi_cols_n, var_name='Date', value_name='NDVI_value').dropna()

In [13]:
tur_plus_mean = tur_by_date.merge(tur[["mean_NDVI", "stdv_NDVI", "cropland", "trees", "grassland", "shrub", "Unnamed: 0"]], how = "left", left_on = "Unnamed: 0", right_on = "Unnamed: 0")
nar_plus_mean = nar_by_date.merge(nar[["mean_NDVI", "stdv_NDVI", "cropland", "trees", "grassland", "shrub", "Unnamed: 0"]], how = "left", left_on = "Unnamed: 0", right_on = "Unnamed: 0")

In [14]:
merge_cols = [col for col in tur.columns if "ppt" in col or col in ["Unnamed: 0"]]

In [15]:
tur_plus_mean.name = "tur_plus_mean"
nar_plus_mean.name = "nar_plus_mean"

#use the full set
base_sets = [tur_plus_mean, nar_plus_mean]

#pull unique dates for each 
dates = [list(tur_plus_mean.Date.unique()), list(nar_plus_mean.Date.unique())]

In [16]:
arguments_t = [(base_sets[0], date) for date in dates[0]]
arguments_n = [(base_sets[1], date) for date in dates[1]]
arguments = arguments_t + arguments_n

In [17]:
# PARALLEL SPLIT DATA BY REGION AND DATE
if __name__ == '__main__':

    with Pool(14) as pool:
        result = []
        for _ in tqdm.tqdm(pool.istarmap(get_dataframes, arguments),
                           total=len(arguments)):
            result.append(_)
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 526/526 [24:31<00:00,  2.80s/it]


In [18]:
tur_date_dfs = result[:420]
nar_date_dfs = result[420:]

In [23]:
# prep for pulling merge specific columns for each date
ppt_arg_t = [(tur, date) for date in dates[0]]
ppt_arg_n = [(nar, date) for date in dates[1]]

In [24]:
arguments = ppt_arg_t
# PARALLEL PULL MERGE DFS FOR EACH REGION X DATE
if __name__ == '__main__':

    with Pool(14) as pool:
        result_t = []
        for _ in tqdm.tqdm(pool.istarmap(make_merge_dfs, arguments),
                           total=len(arguments)):
            result_t.append(_)
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 420/420 [32:07<00:00,  4.59s/it]


In [25]:
arguments = ppt_arg_n
# PARALLEL PULL MERGE DFS FOR EACH REGION X DATE
if __name__ == '__main__':

    with Pool(14) as pool:
        result_n = []
        for _ in tqdm.tqdm(pool.istarmap(make_merge_dfs, arguments),
                           total=len(arguments)):
            result_n.append(_)
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [07:39<00:00,  4.33s/it]


In [26]:
final_arg_t = list(zip(tur_date_dfs, result_t))
final_arg_n = list(zip(nar_date_dfs, result_n))

In [27]:
arguments = final_arg_t
# PARALLEL PULL MERGE DFS FOR EACH REGION X DATE
if __name__ == '__main__':

    with Pool(14) as pool:
        final_t = []
        for _ in tqdm.tqdm(pool.istarmap(merge_and_calc_ppt, arguments),
                           total=len(arguments)):
            final_t.append(_)
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 420/420 [00:29<00:00, 14.06it/s]


In [28]:
arguments = final_arg_n
# PARALLEL PULL MERGE DFS FOR EACH REGION X DATE
if __name__ == '__main__':

    with Pool(14) as pool:
        final_n = []
        for _ in tqdm.tqdm(pool.istarmap(merge_and_calc_ppt, arguments),
                           total=len(arguments)):
            final_n.append(_)
            pass

100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [00:12<00:00,  8.38it/s]


In [29]:
FINAL_T = pd.concat(final_t)
FINAL_N = pd.concat(final_n)

In [30]:
FINAL_T.to_csv("TURKANA_ANOMALIES.csv")
FINAL_N.to_csv("NAROK_ANOMALIES.csv")